#### Завантаження файлів і необхідних бібліотек та csv файлів

In [102]:
from IPython.display import display
import pandas as pd
df = pd.read_csv('./adult.data.csv')
from IPython.core.interactiveshell import InteractiveShell


InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)

#### Перевірка 1, 2, 3

In [109]:
display(df.head(10))


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


#### 1. Скільки чоловіків і жінок (ознака sex) представлено в цьому наборі даних?

In [110]:
df.groupby('sex')['sex'].value_counts()


# df.groupby('sex').value_counts()

sex
Female    10771
Male      21790
Name: count, dtype: int64

#### 2. Який середній вік (ознака age) жінок?

In [111]:
female_data = df[df['sex'] == 'Female']

female_data['age'].mean()


36.85823043357163

#### 3. Яка частка громадян Німеччини (ознака native-country)?

In [112]:
germany_count = df[df['native-country'] == 'Germany'].shape[0]

total_count = df.shape[0]

share_German = germany_count/total_count

share_German

0.004207487485028101

#### 4-5. Які середні значення і середні відхилення віку тих, хто отримує більше 50K в рік (ознака *salary) і тих, хто отримує менше 50K в рік?

In [113]:
filter_data = df[(df['salary'] == '<=50K') | (df['salary'] == '>50K')]

group_salary = filter_data.groupby('salary')

result = group_salary['age'].agg(['mean', 'std'])

result



,mean,std
salary,,
<=50K,36.783738,14.020088
>50K,44.249841,10.519028


#### 6. Чи правда, що люди, які отримують більше 50k, мають як мінімум вищу освіту? (ознака education – Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters чи Doctorate)

In [114]:
# на це питання дам відповідь формою відображення даних, в яких якщо система знайде хоч одне значення 
# people(salary >50K && (education !== Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters чи Doctorate))
# якщо хоч одну таку людину знайду, що заробляє 50К та має освіту нижче Bachelor, то це неправда, в нашому випадку якщо серед переліку кінцевий резултат буде count() > 0, то це неправда

desired_education = ['Bachelors', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', 'Masters', 'Doctorate']

filter_lower_education = df[~df['education'].isin(desired_education)]

filter_50K = df[df['salary'] == '>50K']

filter_50K.shape[0]

# таких людей із освітою нижче університетською 7841 значаить це неправда, що люди, які отримують більше 50k, мають як мінімум вищу освіту


7841

 #### 7. Вивести статистику віку для кажної раси (ознака race) і кожної статі. Використовуйте groupby і describe. Знайдіть таким чином максимальний вік чоловіків раси Amer-Indian-Eskimo.

In [118]:
# Щоб дістатися до потрібних даних max для Amer-Indian-Eskimo, групуємо за віком та гендером

grouped_data = df.groupby(['race', 'sex'])['age'].describe()

# За допомогою loc дістаємось до даних де рядки є в групі:  Amer-Indian-Eskimo та Male і мають колонку max

max_age_amer_indian_male = grouped_data.loc[('Amer-Indian-Eskimo', 'Male'), 'max']

max_age_amer_indian_male




82.0

#### 8. Серед кого більша частка заробляючих багато (>50K): серед одружених чи холостих чоловіків (ознака *marital-status*)? 
#### Одруженими вважаємо тих, у кого *marital-status* починається з *Married* 
#### (Married-civ-spouse, Married-spouse-absent чи Married-AF-spouse), решту вважаємо холостими.**

In [119]:
# Фільтр: лише чоловіки

sex_male_much50K = df[(df['sex'] == 'Male') & (df['salary'] == '>50K')]


# Створюємо новий стовпець для групування на групи холостий-одружений "Status"
sex_male_much50K['Status'] = sex_male_much50K['marital-status'].apply(lambda name: 'Married' if name[:7] == 'Married' else 'Bachelor')

# Групування на холостих та одружених
group_married = sex_male_much50K.groupby(['Status'])['salary'].describe()

group_married.loc[['Bachelor', 'Married'], 'freq']

# Статистика даних показала, що одружені мають більшу частку зарплати вище 50К ніж холостяки





C:\Users\Богдан\AppData\Local\Temp\ipykernel_5644\1280627193.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sex_male_much50K['Status'] = sex_male_much50K['marital-status'].apply(lambda name: 'Married' if name[:7] == 'Married' else 'Bachelor')


Status
Bachelor     697
Married     5965
Name: freq, dtype: object

#### 9. Яку максимальну кількість годин людина працює за тиждень (ознака hours-per-week)? Скільки людей працюють таку кількість годин і який серед них відсоток заробляючих багато?

In [120]:
#Значення макс кількість годин, що є в таблиці
maxWork = df['hours-per-week'].max()

#Фільтр працюючих понаднормово
totalMaxWork = (df[df['hours-per-week'] == maxWork])

countTotalHour = totalMaxWork['hours-per-week'].count()


#Фільтр працюючих понаднормово і заробляють більше 50К
totalMaxWork50K =(df[(df['hours-per-week'] == maxWork) & (df['salary'] == '>50K')])

countTotalMaxWork50K = totalMaxWork50K['hours-per-week'].count()

#Відсоток, хто працює понаднормово та заробляє більше 50К

result= (countTotalMaxWork50K/countTotalHour)*100

result


29.411764705882355

#### 10. Підрахуйте середній час роботи (hours-per-week) заробляючих мало і багато (salary) для кожної країни (native-country).

In [108]:
#Групування, хто заробляє багато, а хто мало по кожній країні та вивід середнього значення за hours-per-week

result = df.groupby(['salary', 'native-country'])['hours-per-week'].mean()

display(result)

salary  native-country            
<=50K   ?                             40.164760
        Cambodia                      41.416667
        Canada                        37.914634
        China                         37.381818
        Columbia                      38.684211
        Cuba                          37.985714
        Dominican-Republic            42.338235
        Ecuador                       38.041667
        El-Salvador                   36.030928
        England                       40.483333
        France                        41.058824
        Germany                       39.139785
        Greece                        41.809524
        Guatemala                     39.360656
        Haiti                         36.325000
        Holand-Netherlands            40.000000
        Honduras                      34.333333
        Hong                          39.142857
        Hungary                       31.300000
        India                         38.233333
     